Import libraries

In [16]:
import pandas as pd
import geopandas as gpd # handle geospatial data

Import Datasets

In [17]:
auto_thefts = gpd.read_file(r'./Data/Auto_Theft_Open_Data_721974690332531929.geojson')
print('Data imported')

Data imported


In [22]:
auto_thefts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7865 entries, 0 to 7864
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OBJECTID      7865 non-null   int64   
 1   VEH_YEAR      7134 non-null   float64 
 2   VEH_MAKE      7280 non-null   object  
 3   VEH_MODEL     4503 non-null   object  
 4   VEH_STYLE     7865 non-null   object  
 5   VEH_COLOUR    7652 non-null   object  
 6   VEH_VALUE     1697 non-null   float64 
 7   WEEKDAY       7865 non-null   object  
 8   RECOVERED     7865 non-null   object  
 9   NB_NAME_EN    7865 non-null   object  
 10  WARD          7865 non-null   object  
 11  SECTOR        7865 non-null   object  
 12  REP_DATE      7865 non-null   object  
 13  OCC_DATE      7865 non-null   object  
 14  YEAR          7865 non-null   int64   
 15  INTERSECTION  7849 non-null   object  
 16  DIVISION      7865 non-null   object  
 17  CENSUS_TRC    7862 non-null   object  
 18  

In [21]:
# Preview the data
auto_thefts.head(2)

,OBJECTID,VEH_YEAR,VEH_MAKE,VEH_MODEL,VEH_STYLE,VEH_COLOUR,VEH_VALUE,WEEKDAY,RECOVERED,NB_NAME_EN,...,OCC_DATE,YEAR,INTERSECTION,DIVISION,CENSUS_TRC,TOD,COUNCILLOR,REP_HOUR,OCC_HOUR,geometry
0,1,2012.0,"TOYOTA/ TOYOTA MOTOR CO., LTD.",CAMRY,Automobile,DGR,8000.0,Sunday,Y,Orléans North West,...,"Sun, 19 May 2019 05:00:00 GMT",2019,"JEANNE D'ARC BLVD, MILLWOOD CRT",East,5050124.04,Evening,Laura Dudas,1100.0,2300.0,POINT Z (-75.53576 45.47940 0.00000)
1,2,2016.0,NISSAN,None,Automobile,BLK,NaN,Monday,Y,Orléans Village - Chateauneuf,...,"Mon, 20 May 2019 05:00:00 GMT",2019,"BELCOURT BLVD, LEBLANC DR, BARSONA PL",East,5050125.05,Night,Laura Dudas,400.0,400.0,POINT Z (-75.52007 45.46870 0.00000)


Data Cleaning

In [ ]:
df_auto = auto_thefts.drop(['CENSUS_TRC','COUNCILLOR'], axis =1)

# Ensure 'geometry' column is in string format
df_auto['geometry'] = df_auto['geometry'].astype(str)

# Extract the lat and long using string manipulation
df_auto['Latitude'] = df_auto['geometry'].str.extract(r'POINT Z \([-+]?[0-9]*\.?[0-9]+ ([+-]?[0-9]*\.?[0-9]+)')
df_auto['Longitude'] = df_auto['geometry'].str.extract(r'POINT Z \(([+-]?[0-9]*\.?[0-9]+)')

# Drop geometry column
df_auto.drop('geometry', axis=1, inplace=True)

# Convert the extracted columns to floats
df_auto['Longitude'] = df_auto['Longitude'].astype(float)
df_auto['Latitude'] = df_auto['Latitude'].astype(float)


# clean the date column to extract just the short date
df_auto['OCC_DATE'] = df_auto['OCC_DATE'].str.replace('GMT','').str.strip()
df_auto['REP_DATE'] = df_auto['REP_DATE'].str.replace('GMT','').str.strip()
df_auto['OCC_DATE'] = pd.to_datetime(df_auto['OCC_DATE'], format='%a, %d %b %Y %H:%M:%S').dt.strftime('%Y-%m-%d')
df_auto['REP_DATE'] = pd.to_datetime(df_auto['REP_DATE'], format='%a, %d %b %Y %H:%M:%S').dt.strftime('%Y-%m-%d')

# get occurance year
df_auto['OCC_DATE'] = pd.to_datetime(df_auto['OCC_DATE'])
df_auto['OCC_YEAR'] = df_auto['OCC_DATE'].dt.year # extract the year
df_auto['OCC_MONTH'] = df_auto['OCC_DATE'].dt.month_name() # extract the month

# by vehicle make/model and year
df_auto['VEH_DESCRIPTION'] = df_auto['VEH_YEAR'].astype(str) + ' ' + df_auto['VEH_MAKE'].astype(str) + ' ' + df_auto['VEH_MODEL'].astype(str)

# filter for auto thefts that occured from 2018
df_auto = df_auto[df_auto['OCC_DATE']>= '2018-01-01'].reset_index(drop=True)

df_auto.head()

C:\Users\Razak\AppData\Local\Temp\ipykernel_5224\84493364.py:4: UserWarning: Geometry column does not contain geometry.
  df_auto['geometry'] = df_auto['geometry'].astype(str)


,OBJECTID,VEH_YEAR,VEH_MAKE,VEH_MODEL,VEH_STYLE,VEH_COLOUR,VEH_VALUE,WEEKDAY,RECOVERED,NB_NAME_EN,...,INTERSECTION,DIVISION,TOD,REP_HOUR,OCC_HOUR,Latitude,Longitude,OCC_YEAR,OCC_MONTH,VEH_DESCRIPTION
0,1,2012.0,"TOYOTA/ TOYOTA MOTOR CO., LTD.",CAMRY,Automobile,DGR,8000.0,Sunday,Y,Orléans North West,...,"JEANNE D'ARC BLVD, MILLWOOD CRT",East,Evening,1100.0,2300.0,45.479399,-75.535764,2019,May,"2012.0 TOYOTA/ TOYOTA MOTOR CO., LTD. CAMRY"
1,2,2016.0,NISSAN,None,Automobile,BLK,NaN,Monday,Y,Orléans Village - Chateauneuf,...,"BELCOURT BLVD, LEBLANC DR, BARSONA PL",East,Night,400.0,400.0,45.468702,-75.520075,2019,May,2016.0 NISSAN None
2,3,NaN,None,None,Other,RED,2500.0,Monday,N,Britannia Village,...,"140 E OF POULIN AVE, RICHMOND RD",West,Evening,1000.0,2300.0,45.362930,-75.790259,2019,May,nan None None
3,4,2018.0,AUDI,None,Automobile,BLK,128000.0,Friday,Y,Vanier South,...,"CAMIL ST, ST. LAURENT BLVD",Central,None,1400.0,NaN,45.440321,-75.646428,2019,April,2018.0 AUDI None
4,5,2008.0,VOLVO,None,Automobile,SIL,NaN,Tuesday,Y,Centretown,...,"BANK ST, LAURIER AVE",Central,Morning,1200.0,1000.0,45.418534,-75.699061,2019,May,2008.0 VOLVO None


Export data for analysis

In [15]:
df_auto.to_csv('Auto Theft Clean Data.csv', index=False)

KPIS

Visualization
- Theft by Vehicle Make
- Thefts of Vehicle by Year
- Theft by Month
- Theft by Day of Week
- Theft by Neighbourhoods
- Theft by OPS Sector
- Theft by City Wards
- Theft by Neighboorhood
